In [1]:
import feedparser
import pandas as pd

# Major Pipleline Functions

def acquire(url, rdata):
    response = feedparser.parse(url)
    rdata.append(response.entries)
    return rdata

def wrangle(rdata):
    # Initialize list to hold post data from 'rss_data'
    post_data = []
    # Convert each RSS entry to a DataFrame and append to list
    for entry in rdata:
        df = pd.DataFrame(entry)
        df = add_source(df)
        #df = add_time(df)
        post_data.append(df)
    # Merge list of dataFrames together with concat
    post_data = pd.concat(post_data, axis=0, sort=False).reset_index()
    return post_data

def format_data(post_data):   
    return post_data[['source', 'title', 'link']]

def visualize(df):
    return df

#def export(df):
    #return df

In [2]:
# Secondary Functions

# Add a column "SOURCE" of constant strings to a DataFrame
def add_source(df):       
    if df['id'][0].startswith("https://www.reddit"):
        df['source'] = "REDDIT"
    elif df['id'][0].startswith("https://news.ycomb"):
        df['source'] = "HNEWS"
    else:
        df['source'] = "INSERT NEW SOURCE HERE"
    return df

In [3]:
def reader (rss_urls):
    # Initialize list to hold the RSS data entries from each URL
    rss_data = []
    for url in rss_urls:       
        rss_data = acquire(url, rss_data)
    print ('Number of RSS_Data Sources:', len(rss_data))
    # Convert RSS entries to a single DataFrame containing all posts
    posts = wrangle(rss_data)
    # Format list of posts into a human-readable table
    return format_data(posts)

urls = ['https://www.reddit.com/r/movies/new.rss', 'https://hnrss.org/newest']
reader(urls)

# TODO: Add column of unformated time strings to the DataFrame

Number of RSS_Data Sources: 2


,source,title,link
0,REDDIT,Venom and Spider-Man crossover movie will be ‘...,https://www.reddit.com/r/movies/comments/c6qbg...
1,REDDIT,Fox's 'Alien Nation' remake has been paused/ca...,https://www.reddit.com/r/movies/comments/c6q5u...
2,REDDIT,Whats The name of this movie,https://www.reddit.com/r/movies/comments/c6q51...
3,REDDIT,The complete theatrical release schedule for J...,https://www.reddit.com/r/movies/comments/c6q43...
4,REDDIT,New and Improved,https://www.reddit.com/r/movies/comments/c6q2x...
5,REDDIT,Cool spanish horror movies?,https://www.reddit.com/r/movies/comments/c6q1o...
6,REDDIT,Lionsgate Picks Up Colin Farrell's Sci-Fi Thri...,https://www.reddit.com/r/movies/comments/c6pyp...
7,REDDIT,A behind-the-scenes look into the recently can...,https://www.reddit.com/r/movies/comments/c6pun...
8,REDDIT,Help me find movie title!,https://www.reddit.com/r/movies/comments/c6psq...
9,REDDIT,I need help finding an animated movie about owls,https://www.reddit.com/r/movies/comments/c6pqd...


In [ ]:
# TODO: Complete these functions, to format time

# Convert Reddit's time format to time_struct
# TODO: Make this function work
def format_reddit_time(time_str):
    return time_str
    
# Convert time strings to time_struct with datetime
# TODO: Make this function work
def add_time(df):
    if df['source'][0] == "REDDIT":
        return df
    else:
        return df

In [ ]:
# How to iterate over rows in a dataframe
# DONT RUN THIS
    for i, row in df.iterrows():
        ifor_val = something
        if <condition>:
            ifor_val = something_else
        df.at[i,'ifor'] = ifor_val